In [1]:
%env DGLBACKEND=tensorflow

env: DGLBACKEND=tensorflow


In [2]:
import dgl
import tensorflow as tf
import tensorflow_addons as tfa
from dgl.nn.tensorflow import GraphConv

2023-09-11 20:00:54.628976: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 20:00:54.666758: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 20:00:54.667515: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 20:00:55.580039: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-11 20:00:56.557685: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for ho

In [3]:
dataset = dgl.data.CoraGraphDataset()

Extracting file to /home/sann-htet/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [4]:
class NodeClassifier(tf.keras.Model):
    def __init__(self, g, in_feats, h_feats, num_classes):
        super(NodeClassifier, self).__init__()
        self.g = g
        self.conv1 = GraphConv(in_feats, h_feats, activation=tf.nn.relu)
        self.conv2 = GraphConv(h_feats, num_classes)
        
    def call(self, in_feat):
        h = self.conv1(self.g, in_feat)
        h = self.conv2(self.g, h)
        return h
    
g = dataset[0]
model = NodeClassifier(g, g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [5]:
def set_gup_if_avaliable():
    device="/cpu:0"
    gpus = tf.config.list_physical_devices("GPU")
    if len(gpus) > 0:
        device = gpus[0]
    return device

device = set_gup_if_avaliable()
g = g.to(device)

In [6]:
def do_eval(model, features, labels, mask):
    logits = model(features, training=False)
    logits = logits[mask]
    labels = labels[mask]
    preds = tf.math.argmax(logits, axis=1)
    acc = tf.reduce_mean(tf.cast(preds == labels, dtype=tf.float32))
    return acc.numpy().item()

In [7]:
NUM_HIDDEN = 16
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 5e-4
NUM_EPOCHS = 200

with tf.device(device):
    feats = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    in_feats = feats.shape[1]
    n_classes = dataset.num_classes
    n_edges = dataset[0].number_of_edges()
    
    model = NodeClassifier(g, in_feats, NUM_HIDDEN, n_classes)
    loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tfa.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    best_val_acc, best_test_acc = 0, 0
    history = []
    for epoch in range(NUM_EPOCHS):
        with tf.GradientTape() as tape:
            logits = model(feats)
            loss = loss_fcn(labels[train_mask], logits[train_mask])
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        val_acc = do_eval(model, feats, labels, val_mask)
        history.append((epoch + 1, loss.numpy().item(), val_acc))
        
        if epoch % 10 == 0:
            print("Epoch {:3d} | train loss: {:.3f} | val acc: {:.3f}".format(epoch, loss.numpy().item(), val_acc))

Epoch   0 | train loss: 1.945 | val acc: 0.186
Epoch  10 | train loss: 1.791 | val acc: 0.516
Epoch  20 | train loss: 1.551 | val acc: 0.610
Epoch  30 | train loss: 1.248 | val acc: 0.656
Epoch  40 | train loss: 0.925 | val acc: 0.686
Epoch  50 | train loss: 0.638 | val acc: 0.722
Epoch  60 | train loss: 0.423 | val acc: 0.748
Epoch  70 | train loss: 0.282 | val acc: 0.762
Epoch  80 | train loss: 0.193 | val acc: 0.774
Epoch  90 | train loss: 0.138 | val acc: 0.782
Epoch 100 | train loss: 0.103 | val acc: 0.780
Epoch 110 | train loss: 0.079 | val acc: 0.782
Epoch 120 | train loss: 0.064 | val acc: 0.780
Epoch 130 | train loss: 0.052 | val acc: 0.778
Epoch 140 | train loss: 0.044 | val acc: 0.774
Epoch 150 | train loss: 0.038 | val acc: 0.776
Epoch 160 | train loss: 0.033 | val acc: 0.776
Epoch 170 | train loss: 0.029 | val acc: 0.778
Epoch 180 | train loss: 0.026 | val acc: 0.776
Epoch 190 | train loss: 0.024 | val acc: 0.778


In [8]:
test_acc = do_eval(model, feats, labels, test_mask)
print("Test acc: {:.3f}".format(test_acc))

Test acc: 0.755
